In [0]:
import nltk

In [0]:
train_stance_url='https://raw.githubusercontent.com/FakeNewsChallenge/fnc-1/master/train_stances.csv'
train_bodies_url='https://raw.githubusercontent.com/FakeNewsChallenge/fnc-1/master/train_bodies.csv'

In [0]:
import pandas as pd
def prepare_data():
    head = pd.read_csv(train_stance_url)
    body = pd.read_csv(train_bodies_url)
    head_array = head.values
    body_array = body.values
    labels = head_array[:,2]
    stance_ids = head_array[:,1]
    body_ids = body_array[:,0]
    new_lab = []
    for i in labels:
        if i == 'unrelated':
            new_lab.append(3)
        if i == 'agree':
            new_lab.append(0)
        if i == 'discuss':
            new_lab.append(2)
        if i == 'disagree':
            new_lab.append(1)
    return {'head':head_array,'body':body_array,'labels':new_lab,}
    
#     pHead, pBody = preprocess(head_array[:,0], body_array[:,1])

In [0]:
data= prepare_data()

In [7]:
len(data['head'])

49972

In [0]:
import numpy as np
h= np.array(data['head'])

In [9]:
print(h.shape)

(49972, 3)


In [10]:
h[0:1]

array([["Police find mass graves with at least '15 bodies' near Mexico town where 43 students disappeared after police clash",
        712, 'unrelated']], dtype=object)

In [0]:
b= np.array(data['body'])

In [12]:
b[0:1]

array([[0,
        'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or w

In [13]:
data['labels'][0:1]

[3]

In [14]:
h[0:5]

array([["Police find mass graves with at least '15 bodies' near Mexico town where 43 students disappeared after police clash",
        712, 'unrelated'],
       ['Hundreds of Palestinians flee floods in Gaza as Israel opens dams',
        158, 'agree'],
       ["Christian Bale passes on role of Steve Jobs, actor reportedly felt he wasn't right for part",
        137, 'unrelated'],
       ['HBO and Apple in Talks for $15/Month Apple TV Streaming Service Launching in April',
        1034, 'unrelated'],
       ["Spider burrowed through tourist's stomach and up into his chest",
        1923, 'disagree']], dtype=object)

In [15]:
data['labels'][0:5]

[3, 0, 3, 3, 1]

In [0]:
stances_train = pd.read_csv(train_stance_url)
body_train = pd.read_csv(train_bodies_url)
train = pd.merge(stances_train, body_train, how='left', on='Body ID')

In [17]:
train[0:2]

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...


In [18]:
train.shape

(49972, 4)

In [0]:
dic={'agree':0,'disagree':1,'discuss':2,'unrelated':3}
train['Stance']=train['Stance'].map(dic) 

In [20]:
train[0:1]

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,3,Danny Boyle is directing the untitled film\n\n...


In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
english_stemmer = nltk.stem.SnowballStemmer('english')
token_pattern = r"(?u)\b\w\w+\b"
stopwords = set(nltk.corpus.stopwords.words('english'))

In [0]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for token in tokens:
        stemmed.append(stemmer.stem(token))
    return stemmed

In [0]:
import re
def preprocess_data(line,
                    token_pattern=token_pattern,
                    exclude_stopword=True,
                    stem=True):
    token_pattern = re.compile(token_pattern, flags = re.UNICODE)
    tokens = [x.lower() for x in token_pattern.findall(line)]
    tokens_stemmed = tokens
    if stem:
#         pass
        tokens_stemmed = stem_tokens(tokens, english_stemmer)
    if exclude_stopword:
#         pass
        tokens_stemmed = [x for x in tokens_stemmed if x not in stopwords]

    return tokens_stemmed

In [38]:
preprocess_data('She is running. He ran away')
# train[0:5]

['run', 'ran', 'away']

In [0]:
preprocess_data(train['Headline'])

Concatinating headline and body strings

In [0]:
train['combined']= train['Headline'] +" "+ train['articleBody']

In [40]:
train['combined'][0]

'Police find mass graves with at least \'15 bodies\' near Mexico town where 43 students disappeared after police clash Danny Boyle is directing the untitled film\n\nSeth Rogen is being eyed to play Apple co-founder Steve Wozniak in Sony’s Steve Jobs biopic.\n\nDanny Boyle is directing the untitled film, based on Walter Isaacson\'s book and adapted by Aaron Sorkin, which is one of the most anticipated biopics in recent years.\n\nNegotiations have not yet begun, and it’s not even clear if Rogen has an official offer, but the producers — Scott Rudin, Guymon Casady and Mark Gordon — have set their sights on the talent and are in talks.\n\nOf course, this may all be for naught as Christian Bale, the actor who is to play Jobs, is still in the midst of closing his deal. Sources say that dealmaking process is in a sensitive stage.\n\nInsiders say Boyle will is flying to Los Angeles to meet with actress to play one of the female leads, an assistant to Jobs. Insiders say that Jessica Chastain is

In [41]:
train['Headline'][0]

"Police find mass graves with at least '15 bodies' near Mexico town where 43 students disappeared after police clash"

**Preprocessing** the combined string = Stemming + Stop words removal

Generating **unigrams**= ['this , 'is' , 'a' ,'cat']

In [0]:
unigram=[]
for i in train['combined']:
  unigram.append(preprocess_data(i))

In [0]:
head_unigram=[]
for i in train['Headline']:
  head_unigram.append(preprocess_data(i))

In [0]:
body_unigram=[]
for i in train['articleBody']:
  body_unigram.append(preprocess_data(i))


In [45]:
unigram_np = np.array(unigram)
unigram_np.shape

(49972,)

**Making strings from unigrams** 

In [0]:

unigram_comb=[]
for count,i in enumerate(unigram_np):
  unigram_comb.append(' '.join(unigram_np[count]))

In [0]:
unigram_comb_head=[]
for count,i in enumerate(head_unigram):
  unigram_comb_head.append(' '.join(head_unigram[count]))

In [0]:
unigram_comb_body=[]
for count,i in enumerate(body_unigram):
  unigram_comb_body.append(' '.join(body_unigram[count]))

In [49]:
len(unigram_comb)

49972

In [50]:
len(unigram_comb[0])

749

In [51]:
len(unigram)

49972

In [0]:
train['headline_unigram']=np.array(unigram_comb_head)
train['combined_unigram']=np.array(unigram_comb)
train['body_unigram']=np.array(unigram_comb_body)

In [0]:
vocab_size=5000
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,3), max_df=0.8, min_df=2,max_features=vocab_size)
vec.fit(train["combined_unigram"]) # Tf-idf calculated on the combined training + test set
vocabulary = vec.vocabulary_

In [59]:
vecH = TfidfVectorizer(ngram_range=(1, 3), max_df=0.8, min_df=2, vocabulary=vocabulary)
xHeadlineTfidf = vecH.fit_transform(train['headline_unigram']) # use ' '.join(Headline_unigram) instead of Headline since the former is already stemmed
print ('xHeadlineTfidf.shape:')
print (xHeadlineTfidf.shape)

xHeadlineTfidf.shape:
(49972, 5000)


In [60]:
len(vocabulary)

5000

In [61]:
xHeadlineTfidf[0]

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [62]:
vecB = TfidfVectorizer(ngram_range=(1, 3), max_df=0.8, min_df=2, vocabulary=vocabulary,max_features=vocab_size)
xBodyTfidf = vecB.fit_transform(train['body_unigram'])
print ('xBodyTfidf.shape:')
print (xBodyTfidf.shape)

xBodyTfidf.shape:
(49972, 5000)


In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x= np.hstack((xHeadlineTfidf,xBodyTfidf))


In [52]:
train[0:1]

,Headline,Body ID,Stance,articleBody,combined,headline_unigram,combined_unigram,body_unigram
0,Police find mass graves with at least '15 bodi...,712,3,Danny Boyle is directing the untitled film\n\n...,Police find mass graves with at least '15 bodi...,polic find mass grave least 15 bodi near mexic...,polic find mass grave least 15 bodi near mexic...,danni boyl direct untitl film seth rogen eye p...


In [0]:
train['xHeadlineTfidf']=xHeadlineTfidf
train['xBodyTfidf']=xBodyTfidf
train['combinedTfidf']=x

In [59]:
a = np.array([[1,2],[3,4],[50,51]]) 
b = np.array([[5,6],[7,8],[9,10]])
print(a.shape)
print(b.shape)
c = np.hstack((a,b))
print(c)
print(c.shape)

(3, 2)
(3, 2)
[[ 1  2  5  6]
 [ 3  4  7  8]
 [50 51  9 10]]
(3, 4)


In [60]:
print(xHeadlineTfidf.shape)
print(xBodyTfidf.shape)

(49972, 507813)
(49972, 507813)


In [62]:
x= np.hstack((xHeadlineTfidf,xBodyTfidf))
print(x.shape)

(2,)


In [0]:
xheadline=np.array(xHeadlineTfidf)

In [64]:
type(xHeadlineTfidf)

scipy.sparse.csr.csr_matrix

In [0]:
from scipy.sparse import csr_matrix
xheadline= xHeadlineTfidf.toarray()

In [71]:
xheadline.shape

(49972, 5000)